In [1]:
from qtpg.team import Team
from qtpg.learner import Learner
from qtpg.program import Program
from qtpg.rule import Rule
from qtpg.figure13 import Figure13
from qtpg.figure12 import Figure12
from qtpg.figure9 import Figure9
from qtpg.search_manager import SearchManager
import numpy as np
import uuid
import copy
import matplotlib.pyplot as plt

In [2]:
runs = []

In [3]:
run = 30

In [4]:

# env params
memorySize = 20
legalMove = 0.1
illegalMove = -0.01
outOfBounds = -0.01
memoryRepeat = -0.01
goalReached = 100

# envName = 'Figure 13'
# env = Figure13(5, 5, (2, 4), (0, 0), memorySize, legalMove, illegalMove, outOfBounds, memoryRepeat, goalReached)
# envName = 'Figure 12'
# env = Figure12(5, 5, (4, 4), (0, 0), memorySize, legalMove, illegalMove, outOfBounds, memoryRepeat, goalReached)
envName = 'Figure 9'
env = Figure9(10, 10, (4, 9), (0, 3), memorySize, legalMove, illegalMove, outOfBounds, memoryRepeat, goalReached)

In [5]:
# adam_and_eve = []
# env.reset()
# init_team = Team(uuid.uuid4(), 0, 0, 0, 0, 0)
# # init_rule = Rule(uuid.uuid4(), [0, 0, 0, 0], 0, 0)
# # init_learner = Learner(uuid.uuid4(), init_rule)
# # (init_team.learners).append(init_learner)
# # init_team.mostRecent = init_learner
# adam_and_eve.append(init_team)

# for adam in adam_and_eve:
#     print(adam.learners)
#     print('------')
#     adam.init_search(env, [2, 3])
#     print(adam.learners)
#     for learner in adam.learners:
#         print(learner.program.rule.action_set)
#         print(learner.program.rule.region)

In [ ]:
numGens = 50
maxTeamPool = 5
# init search manager and starting teams
searchManager = SearchManager(maxTeamPool)

# init_team = Team(uuid.uuid4(), 0, 0, 0, 0, 0)
# init_rule = Rule(uuid.uuid4(), [0, 0, 0, 0], 0, 0)
# init_learner = Learner(uuid.uuid4(), init_rule)
# (init_team.learners).append(init_learner)
# init_team.mostRecent = init_learner
# (searchManager.teamPool).append(init_team)

# init teams
adam_and_eve = []
env.reset()
init_team = Team(uuid.uuid4(), 0, 0, 1, 1, 0)
init_team.init_search(env, [2, 3])
adam_and_eve.append(init_team)
win = False
for adam in adam_and_eve:
#     win = adam.search(env)
    data = [adam, win]
    searchManager.evaluate_team(data)

# print('-----init pop start-----')
# for team in searchManager.teamPool:
#     print(f'Team id: {team.id}')
#     print('Learners:')
#     for learner in team.learners:
#         print(learner.id)
# print('-----init pop end-----')

# variables for tracking fitness, for curves
scores = []
gens = []
maxes = []
averages = []
mins = []

for gen in range(numGens):
#     if win:
#         print(f'got to --> gen {gen+1}')
#         break
    toEvaluateAfterGen = []
    scores = []
    print(f'>>>>>>>>>>>>>>>>>>>>>>>Gen {gen+1} start!>>>>>>>>>>>>>>>>>>>>>>>')
    # get it started
#     if gen == 0:
#         for adam in adam_and_eve:
#             win = adam.search(env)
#             data = [adam, win]
#             searchManager.evaluate_team(data)
#     else:
    for parent in searchManager.teamPool:
        # only run if the team has not yet won
        if parent not in searchManager.winners:
            print(f'Parent: {parent.id}')
    #             print(f'Start: {parent.start_state}')
            env.current_state = parent.start_state

            # make offspring from parent, then let that offspring make a new rule
            child = copy.deepcopy(parent)
            child.id = uuid.uuid4()
            win = child.search(env)
            print(f'Child: {child.id}')
            print('Start Child Regions --------')
            for learner in child.learners:
                print(learner.program.rule.region)
            print('End Child Regions --------')
            print(f'Child Fitness: {child.fitness}') 
            toEvaluateAfterGen.append([child, win])
            scores.append(child)
        else:
            print(f'Skipping Team: {parent.id}')
            print(parent.fitness)
            scores.append(parent)
            
    for data in toEvaluateAfterGen:
        searchManager.evaluate_team(data)
    print('%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%')
    print(f'End of Gen {gen+1} Regions')
    for i in range(len(searchManager.teamPool)):
        print(f'Team: {i+1} --> {searchManager.teamPool[i].fitness}')
        for learner in searchManager.teamPool[i].learners:
            print(learner.program.rule.region)
    print(f'>>>>>>>>>>>>>>>>>>>>>>>Gen {gen+1} end!>>>>>>>>>>>>>>>>>>>>>>>')
    # start: manage fitness curves (at end of gen)
    gens.append(gen)
    total = 0
    max_fitness = -100
    min_fitness = 100

    for team in scores:
        total += team.fitness
        if team.fitness > max_fitness:
            max_fitness = team.fitness
        if team.fitness < min_fitness:
            min_fitness = team.fitness
    average_fitness = total / len(searchManager.teamPool)
    maxes.append(max_fitness)
    averages.append(average_fitness)
    mins.append(min_fitness)
    runs.append({'run': run, 'maxes': maxes, 'averages': averages, 'mins': mins})
    # end: manage fitness curves (at end of gen)
for team in searchManager.winners:
    print('$$$$$$$$$$$$$ winnnnnner start $$$$$$$$$$$$$$$$$$$$')
    print(team.fitness)
    print('Learners:')
    for learner in team.learners:
        print(f'Region: {learner.program.rule.region} --> Action: {learner.program.rule.action_set}')
    print('$$$$$$$$$$$$$ winnnnnner end $$$$$$$$$$$$$$$$$$$$')

new step-------
2
(0, 3)
new step-------
2
(0, 4)
new step-------
2
(0, 4)
new step-------
3
(0, 3)
new step-------
3
(0, 2)
new step-------
3
(0, 1)
new step-------
3
(0, 0)
new step-------
3
(0, 0)
[0, 0, 0, 4]
>>>>>>>>>>>>>>>>>>>>>>>Gen 1 start!>>>>>>>>>>>>>>>>>>>>>>>
Parent: bba37682-aff0-4677-83e1-d4ae4a2fa702
Start: (0, 3)
New real step: 
0
(1, 3)
New real step: 
0
(2, 3)
New real step: 
0
(3, 3)
New real step: 
0
(3, 3)
New real step: 
1
(2, 3)
New real step: 
1
(1, 3)
New real step: 
1
(0, 3)
New real step: 
1
(0, 3)
Child: 10b1cba2-dcaa-4dbb-b775-9b82964c669b
Start Child Regions --------
[0, 0, 0, 2]
[0, 0, 4, 4]
[1, 3, 0, 3]
End Child Regions --------
Child Fitness: 1.03
%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
End of Gen 1 Regions
Team: 1 --> 0.48
[0, 0, 0, 4]
Team: 2 --> 1.03
[0, 0, 0, 2]
[0, 0, 4, 4]
[1, 3, 0, 3]
>>>>>>>>>>>>>>>>>>>>>>>Gen 1 end!>>>>>>>>>>>>>>>>>>>>>>>
>>>>>>>>>>>>>>>>>>>>>>>Gen 2 start!>>>>>>>>>>>>>>>>>>>>>>>
Parent: bba37682-aff0-

[0, 6, 1, 1]
[0, 6, 3, 5]
[1, 2, 5, 4]
[1, 2, 6, 9]
[0, 5, 4, 6]
[0, 5, 8, 7]
[1, 7, 0, 0]
[1, 7, 2, 3]
[0, 1, 6, 7]
[0, 1, 9, 9]
[1, 8, 0, 2]
End Child Regions --------
Child Fitness: 5.66
Parent: bff582d9-d6fa-4ff6-9416-b222ca21da94
Start: (4, 1)
New real step: 
2
(4, 1)
Backtrackregioncehck: [1, 1, 0, 3]
New real step: 
2
(2, 2)
New real step: 
2
(2, 3)
New real step: 
2
(2, 4)
New real step: 
2
(2, 4)
Child: 393660b9-5897-4f56-ba8c-1aa72248fa71
Start Child Regions --------
[0, 0, 1, 4]
[1, 0, 0, 1]
[1, 0, 5, 9]
[0, 2, 0, 0]
[0, 2, 2, 4]
[1, 1, 8, 7]
[1, 1, 9, 9]
[0, 8, 0, 0]
[0, 8, 2, 5]
[1, 1, 1, 0]
[0, 0, 0, 0]
[0, 0, 4, 4]
[1, 1, 0, 1]
[1, 1, 5, 8]
[0, 2, 1, 4]
End Child Regions --------
Child Fitness: 5.34
Parent: 5c556403-5ee5-44a1-aeeb-26b3d2521ca1
Start: (0, 0)
New real step: 
3
(0, 0)
Backtrackregioncehck: [1, 0, 1, 1]
New real step: 
3
(2, 0)
Child: c99eb67e-d5a1-475c-ad06-85148e02d34c
Start Child Regions --------
[0, 0, 0, 2]
[0, 0, 4, 4]
[1, 3, 1, 3]
[0, 0, 0, 0]
[0, 0, 

[0, 2, 1, 4]
[1, 0, 0, 0]
[1, 0, 3, 6]
[0, 0, 0, 4]
End Child Regions --------
Child Fitness: 7.95
Parent: 10f412d9-d415-408c-be9b-7b4b4de1dc05
Start: (9, 5)
New real step: 
2
(9, 6)
New real step: 
2
(9, 7)
New real step: 
2
(9, 8)
New real step: 
2
(9, 9)
New real step: 
2
(9, 9)
New real step: 
3
(9, 8)
New real step: 
3
(9, 7)
New real step: 
3
(9, 6)
New real step: 
3
(9, 5)
New real step: 
3
(9, 4)
Child: 6fba6669-e040-4c02-8099-4c950c01f8fe
Start Child Regions --------
[0, 0, 0, 2]
[0, 0, 4, 4]
[1, 3, 1, 3]
[0, 0, 0, 0]
[0, 0, 2, 4]
[1, 1, 0, 1]
[1, 1, 3, 9]
[0, 2, 1, 4]
[1, 0, 8, 6]
[0, 7, 1, 5]
[1, 0, 1, 1]
[0, 0, 1, 4]
[1, 0, 0, 7]
[1, 0, 9, 9]
[0, 8, 2, 4]
[0, 8, 6, 5]
[1, 5, 5, 8]
[0, 9, 5, 9]
End Child Regions --------
Child Fitness: 8.71
%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
End of Gen 14 Regions
Team: 1 --> 8.549999999999999
[0, 0, 0, 2]
[0, 0, 4, 4]
[1, 3, 1, 3]
[0, 0, 0, 0]
[0, 0, 2, 4]
[1, 1, 0, 1]
[1, 1, 3, 9]
[0, 2, 1, 4]
[1, 0, 8, 6]
[0, 

New real step: 
2
(7, 1)
New real step: 
2
(7, 2)
New real step: 
2
(7, 3)
New real step: 
2
(7, 4)
New real step: 
2
(7, 5)
New real step: 
2
(7, 5)
Child: e18d5aba-9915-4cc1-a014-8f1fc8ab87ab
Start Child Regions --------
[0, 0, 0, 2]
[0, 0, 4, 4]
[1, 3, 1, 3]
[0, 0, 0, 0]
[0, 0, 2, 4]
[1, 1, 0, 1]
[1, 1, 3, 9]
[0, 2, 1, 4]
[1, 0, 8, 6]
[0, 7, 1, 5]
[1, 0, 1, 1]
[0, 0, 1, 4]
[1, 0, 0, 7]
[1, 0, 9, 9]
[0, 8, 2, 1]
[0, 8, 3, 5]
[1, 2, 5, 6]
[1, 2, 8, 9]
[0, 7, 0, 1]
[0, 7, 3, 5]
[1, 2, 5, 5]
[1, 2, 7, 9]
[0, 6, 0, 1]
[0, 6, 3, 5]
[1, 2, 5, 6]
[1, 2, 8, 9]
[0, 7, 0, 5]
End Child Regions --------
Child Fitness: 11.02
Parent: 0a2064c1-90bf-42f9-b455-b8b478b7e0b1
Start: (7, 2)
New real step: 
1
(6, 2)
New real step: 
1
(5, 2)
New real step: 
1
(5, 2)
New real step: 
0
(6, 2)
New real step: 
0
(7, 2)
New real step: 
0
(8, 2)
New real step: 
0
(9, 2)
New real step: 
0
(9, 2)
Child: 0fc4995f-c081-4bd8-9cd7-c7c6e2897273
Start Child Regions --------
[0, 0, 0, 2]
[0, 0, 4, 4]
[1, 3, 1, 3]
[0, 0, 

(6, 5)
New real step: 
0
(7, 5)
New real step: 
0
(8, 5)
New real step: 
0
(9, 5)
New real step: 
0
(9, 5)
Child: 9271359d-4c4b-47ed-9e47-68a113d4bdb3
Start Child Regions --------
[0, 0, 0, 2]
[0, 0, 4, 4]
[1, 3, 0, 2]
[1, 3, 4, 3]
[0, 3, 0, 0]
[0, 3, 4, 4]
[1, 1, 3, 5]
[1, 1, 7, 9]
[0, 6, 1, 1]
[0, 6, 3, 5]
[1, 2, 5, 4]
[1, 2, 6, 9]
[0, 5, 4, 6]
[0, 5, 8, 7]
[1, 7, 0, 1]
[1, 7, 3, 3]
[0, 2, 6, 5]
[0, 2, 7, 9]
[1, 6, 0, 4]
[1, 6, 6, 5]
[0, 5, 0, 1]
[0, 5, 3, 2]
[1, 2, 5, 5]
[1, 2, 7, 9]
[0, 6, 0, 4]
[0, 6, 6, 5]
[1, 5, 5, 4]
[1, 5, 6, 9]
[0, 5, 0, 1]
[0, 5, 3, 3]
[1, 2, 5, 5]
[1, 2, 7, 9]
[0, 6, 0, 2]
[0, 6, 4, 5]
[1, 3, 5, 5]
[1, 3, 7, 9]
[0, 6, 0, 4]
[0, 6, 6, 5]
[1, 5, 5, 9]
End Child Regions --------
Child Fitness: 13.000000000000002
Parent: ff5397cc-acee-4f44-8a7c-12f107436ca9
Start: (7, 5)
New real step: 
2
(7, 5)
Backtrackregioncehck: [1, 5, 5, 6]
New real step: 
2
(5, 6)
New real step: 
2
(5, 7)
New real step: 
2
(5, 7)
Child: d9da0b54-4d10-4b0a-9a0a-b78833a6d7cf
Start Child Re

...

[0, 0, 4, 4]
[1, 3, 1, 3]
[0, 0, 0, 0]
[0, 0, 2, 4]
[1, 1, 0, 1]
[1, 1, 3, 9]
[0, 2, 1, 4]
[1, 0, 8, 6]
[0, 7, 1, 5]
[1, 0, 1, 1]
[0, 0, 1, 4]
[1, 0, 0, 7]
[1, 0, 9, 9]
[0, 8, 2, 4]
[0, 8, 6, 5]
[1, 5, 5, 8]
[0, 9, 5, 6]
[0, 9, 8, 9]
[1, 7, 0, 4]
[1, 7, 6, 9]
[0, 5, 1, 1]
[1, 0, 0, 0]
[1, 0, 3, 3]
[0, 0, 0, 0]
[0, 0, 4, 4]
[1, 1, 0, 7]
[1, 1, 9, 8]
[0, 8, 1, 5]
[1, 0, 2, 4]
[1, 0, 6, 9]
[0, 5, 6, 6]
[0, 5, 8, 7]
[1, 7, 0, 0]
[1, 7, 2, 3]
[0, 1, 6, 6]
[0, 1, 8, 9]
[1, 7, 9, 8]
[0, 9, 3, 3]
[0, 9, 5, 9]
[1, 4, 5, 4]
[1, 4, 6, 9]
[0, 5, 0, 2]
[0, 5, 4, 4]
[1, 3, 5, 6]
[1, 3, 8, 9]
[0, 7, 0, 5]
End Child Regions --------
Child Fitness: 19.98
Parent: 5a99a0dc-a82f-4937-a05a-0686f1563953
Start: (6, 1)
New real step: 
0
(7, 1)
New real step: 
0
(8, 1)
New real step: 
0
(9, 1)
New real step: 
0
(9, 1)
New real step: 
1
(8, 1)
New real step: 
1
(7, 1)
New real step: 
1
(6, 1)
New real step: 
1
(5, 1)
New real step: 
1
(4, 1)
New real step: 
1
(3, 1)
Child: baffc85b-0d98-46d4-aeb1-ef18dd64aa6a
S

[1, 3, 5, 5]
[1, 3, 7, 9]
[0, 6, 0, 0]
[0, 6, 2, 5]
[1, 1, 4, 6]
[1, 1, 8, 9]
[0, 7, 1, 3]
[0, 7, 5, 5]
[1, 4, 5, 9]
Team: 2 --> 21.18
[0, 0, 0, 2]
[0, 0, 4, 4]
[1, 3, 1, 3]
[0, 0, 0, 0]
[0, 0, 2, 4]
[1, 1, 0, 1]
[1, 1, 3, 9]
[0, 2, 1, 4]
[1, 0, 8, 6]
[0, 7, 1, 5]
[1, 0, 1, 1]
[0, 0, 1, 4]
[1, 0, 0, 7]
[1, 0, 9, 9]
[0, 8, 2, 4]
[0, 8, 6, 5]
[1, 5, 5, 8]
[0, 9, 5, 6]
[0, 9, 8, 9]
[1, 7, 0, 4]
[1, 7, 6, 9]
[0, 5, 1, 1]
[1, 0, 0, 0]
[1, 0, 3, 3]
[0, 0, 0, 0]
[0, 0, 4, 4]
[1, 1, 0, 7]
[1, 1, 9, 8]
[0, 8, 1, 5]
[1, 0, 2, 4]
[1, 0, 6, 9]
[0, 5, 6, 6]
[0, 5, 8, 7]
[1, 7, 0, 0]
[1, 7, 2, 3]
[0, 1, 6, 6]
[0, 1, 8, 9]
[1, 7, 9, 8]
[0, 9, 3, 3]
[0, 9, 5, 9]
[1, 4, 5, 4]
[1, 4, 6, 9]
[0, 5, 0, 2]
[0, 5, 4, 4]
[1, 3, 5, 6]
[1, 3, 8, 9]
[0, 7, 0, 0]
[0, 7, 2, 5]
[1, 1, 3, 1]
[1, 1, 5, 9]
[0, 2, 0, 4]
Team: 3 --> 20.73
[0, 0, 0, 2]
[0, 0, 4, 4]
[1, 3, 1, 3]
[0, 0, 0, 0]
[0, 0, 2, 4]
[1, 1, 0, 1]
[1, 1, 3, 9]
[0, 2, 1, 4]
[1, 0, 8, 6]
[0, 7, 1, 5]
[1, 0, 1, 1]
[0, 0, 1, 4]
[1, 0, 0, 7]
[1, 0, 9, 9]
[0

[1, 0, 0, 0]
[1, 0, 3, 3]
[0, 0, 0, 0]
[0, 0, 4, 4]
[1, 1, 0, 7]
[1, 1, 9, 8]
[0, 8, 1, 5]
[1, 0, 2, 4]
[1, 0, 6, 9]
[0, 5, 6, 6]
[0, 5, 8, 7]
[1, 7, 0, 0]
[1, 7, 2, 3]
[0, 1, 6, 6]
[0, 1, 8, 9]
[1, 7, 9, 8]
[0, 9, 3, 3]
[0, 9, 5, 9]
[1, 4, 5, 4]
[1, 4, 6, 9]
[0, 5, 0, 2]
[0, 5, 4, 4]
[1, 3, 5, 6]
[1, 3, 8, 9]
[0, 7, 0, 0]
[0, 7, 2, 5]
[1, 1, 3, 5]
[1, 1, 7, 9]
[0, 6, 1, 4]
[0, 6, 6, 5]
[1, 5, 5, 8]
[0, 9, 5, 9]
End Child Regions --------
Child Fitness: 22.82
Parent: cd9c257d-14d4-4b4b-8eab-429d5ae7e0b0
Start: (9, 9)
New real step: 
1
(8, 9)
New real step: 
1
(7, 9)
New real step: 
1
(6, 9)
New real step: 
1
(6, 9)
New real step: 
0
(7, 9)
New real step: 
0
(8, 9)
New real step: 
0
(9, 9)
New real step: 
0
(9, 9)
Backtrackregioncehck: [0, 9, 9, 8]
New real step: 
0
(9, 7)
Child: 8a96c272-b356-4ef4-b78d-2c6ed9adca43
Start Child Regions --------
[0, 0, 0, 2]
[0, 0, 4, 4]
[1, 3, 0, 2]
[1, 3, 4, 3]
[0, 3, 0, 0]
[0, 3, 4, 4]
[1, 1, 3, 5]
[1, 1, 7, 9]
[0, 6, 1, 1]
[0, 6, 3, 5]
[1, 2, 5, 4]
[

[0, 1, 3, 4]
[1, 2, 1, 3]
[0, 0, 0, 1]
[0, 0, 5, 4]
[1, 2, 0, 3]
End Child Regions --------
Child Fitness: 23.37
Parent: 3f019883-d659-457b-abe4-7e1dcebdf918
Start: (7, 5)
New real step: 
2
(7, 5)
Backtrackregioncehck: [1, 5, 6, 6]
New real step: 
2
(5, 6)
New real step: 
2
(5, 7)
New real step: 
2
(5, 7)
Child: 96e06909-defc-4f58-a151-8612fb67041b
Start Child Regions --------
[0, 0, 0, 2]
[0, 0, 4, 4]
[1, 3, 1, 3]
[0, 0, 0, 0]
[0, 0, 2, 4]
[1, 1, 0, 1]
[1, 1, 3, 9]
[0, 2, 1, 4]
[1, 0, 8, 6]
[0, 7, 1, 5]
[1, 0, 1, 1]
[0, 0, 1, 4]
[1, 0, 0, 7]
[1, 0, 9, 9]
[0, 8, 2, 4]
[0, 8, 6, 5]
[1, 5, 5, 8]
[0, 9, 5, 6]
[0, 9, 8, 9]
[1, 7, 0, 4]
[1, 7, 6, 9]
[0, 5, 1, 1]
[1, 0, 0, 0]
[1, 0, 3, 3]
[0, 0, 0, 0]
[0, 0, 4, 4]
[1, 1, 0, 7]
[1, 1, 9, 8]
[0, 8, 1, 5]
[1, 0, 2, 4]
[1, 0, 6, 9]
[0, 5, 6, 6]
[0, 5, 8, 7]
[1, 7, 0, 0]
[1, 7, 2, 3]
[0, 1, 6, 6]
[0, 1, 8, 9]
[1, 7, 9, 8]
[0, 9, 3, 3]
[0, 9, 5, 9]
[1, 4, 5, 4]
[1, 4, 6, 9]
[0, 5, 0, 2]
[0, 5, 4, 4]
[1, 3, 5, 6]
[1, 3, 8, 9]
[0, 7, 0, 0]
[0, 7, 2,

[0, 0, 5, 4]
[1, 2, 0, 1]
[1, 2, 3, 3]
[0, 2, 0, 0]
[0, 2, 2, 4]
[1, 1, 8, 7]
[1, 1, 9, 9]
[0, 8, 1, 4]
[0, 8, 6, 5]
[1, 5, 5, 8]
[0, 9, 0, 2]
[0, 9, 4, 3]
[1, 3, 5, 6]
[1, 3, 8, 9]
[0, 7, 1, 5]
[1, 0, 3, 5]
[1, 0, 9, 9]
[0, 6, 0, 2]
[0, 6, 4, 5]
[1, 3, 5, 5]
[1, 3, 7, 9]
[0, 6, 0, 0]
[0, 6, 2, 5]
[1, 1, 4, 6]
[1, 1, 8, 9]
[0, 7, 1, 3]
[0, 7, 5, 5]
[1, 4, 5, 5]
[1, 4, 7, 9]
[0, 6, 0, 2]
[0, 6, 4, 5]
[1, 3, 5, 8]
[0, 9, 0, 4]
[0, 9, 6, 5]
[1, 5, 5, 8]
[0, 9, 5, 3]
[1, 4, 9, 9]
Team: 3 --> 24.8
[0, 0, 0, 2]
[0, 0, 4, 4]
[1, 3, 1, 3]
[0, 0, 0, 0]
[0, 0, 2, 4]
[1, 1, 0, 1]
[1, 1, 3, 9]
[0, 2, 1, 4]
[1, 0, 8, 6]
[0, 7, 1, 5]
[1, 0, 1, 1]
[0, 0, 1, 4]
[1, 0, 0, 7]
[1, 0, 9, 9]
[0, 8, 2, 4]
[0, 8, 6, 5]
[1, 5, 5, 8]
[0, 9, 5, 6]
[0, 9, 8, 9]
[1, 7, 0, 4]
[1, 7, 6, 9]
[0, 5, 1, 1]
[1, 0, 0, 0]
[1, 0, 3, 3]
[0, 0, 0, 1]
[0, 0, 5, 4]
[1, 2, 0, 1]
[1, 2, 3, 3]
[0, 2, 0, 0]
[0, 2, 2, 4]
[1, 1, 8, 7]
[1, 1, 9, 9]
[0, 8, 1, 4]
[0, 8, 6, 5]
[1, 5, 5, 8]
[0, 9, 0, 2]
[0, 9, 4, 3]
[1, 3, 5, 6]
[1, 3, 8

In [ ]:
#fitness curves
# x = []
# y = []
# for gen in gens:
#     x.append(gen)

# for average in averages:
#     y.append(average)
# plt.xlabel('Generation')
# plt.ylabel('Average Score')
# plt.plot(x, y)
# plt.show()

In [ ]:
# #fitness curves
# x = []
# y = []
# for gen in gens:
#     x.append(gen)

# for max_ in maxes:
#     y.append(max_)
# plt.xlabel('Generation')
# plt.ylabel('Max Score')
# plt.plot(x, y)
# plt.show()

In [ ]:
for team in searchManager.winners:
    print('$$$$$$$$$$$$$ winnnnnner start $$$$$$$$$$$$$$$$$$$$')
    print(team.fitness)
    print('Learners:')
    for learner in team.learners:
        print(f'Region: {learner.program.rule.region} --> Action: {learner.program.rule.action_set}')
    print('$$$$$$$$$$$$$ winnnnnner end $$$$$$$$$$$$$$$$$$$$')

In [ ]:
team = searchManager.winners[0]
for learner in team.learners:
    print(learner.program.rule.region)

for i in range(5):
    print(f'Epoch: {i+1} --------')
    env.reset()
    win = False
    selected_learner = team.select_learner(env)
    while not win:
        win, needs_update, reward, winning_action = team.q_evaluation(env, selected_learner)
        print(reward)
        if not win:
            selected_learner = team.select_learner(env)
            team.transition_update(reward, winning_action, needs_update, selected_learner)

    team.final_update(reward, winning_action, needs_update)

print(win)
for learner in team.learners:
    print(f'{learner.program.rule.region}')
    for i in range(len(learner.program.rule.action_set)):
          print(f'{learner.program.rule.action_set[i]} --> {learner.program.rule.value_set[i]}')

In [ ]:
region_action_scores = []
for learner in team.learners:
    if learner.program.rule.value_set[0] > learner.program.rule.value_set[1]:
        region_action_scores.append({'region': learner.program.rule.region, 'action': learner.program.rule.action_set[0], 'q': learner.program.rule.value_set[0]})
    else:
        region_action_scores.append({'region': learner.program.rule.region, 'action': learner.program.rule.action_set[1], 'q': learner.program.rule.value_set[1]})

sorted_region_action_scores = sorted(region_action_scores, key=lambda value: float(value['q']), reverse=True)
# print(sorted_region_action_scores)

action_states = []
for record in sorted_region_action_scores:
    action = ''
    if record['action'] == 0:
        action = '\u2191'
    elif record['action'] == 1:
        action = '\u2193'
    elif record['action'] == 2:
        action = '\u2192'
    elif record['action'] == 3:
        action = '\u2190'
    
    for i in range(record['region'][3] - record['region'][2]+1):
        state = [0, 0]
        state[record['region'][0]] = record['region'][1]
        state[not record['region'][0]] = record['region'][2] + i
        
        # ensure we do not enter duplicate states
        found = 0
        for pair in action_states:
            if pair['state'] == state:
                found = 1
                
        if found == 0:
            action_states.append({'state': (state[0], state[1]), 'action': action})

# add in the rest of the states, either they are not visited, or illegal
for n in reversed(range(env.rows)):
    for m in range(env.cols):
        action = ''
        found = 0
        for record in action_states:
            if record['state'] == (n, m):
                found = 1
                action = record['action']
        if found == 0:
            if not env.check_legal((n, m)):
                action = 'X'
            else:
                action = '?'
        print(f'{action} ', end='')
    print('\n')

# print(action_states)
    

In [ ]:
# #fitness curves
# x = []
# y = []

# for gen in gens:
#     x.append(gen)

# for i in range(len(runs)):
#     y = []
#     for max_ in runs[i]['maxes']:
#         y.append(max_)
#     plt.xlabel('Generation')
#     plt.ylabel('Max Score')
#     plt.plot(x, y)
# plt.show()

In [ ]:
#fitness curves
x = []
y = []
average_maxes = np.zeros(50)

# for i in range(len(runs)):
#     for max_ in runs[i]['maxes']:
#         average_maxes[i] += max_
#     average_maxes[i] /= len(runs[i])

for i in range(len(gens)):
    for q in range(len(runs)):
        average_maxes[i] += runs[q]['maxes'][i]
    average_maxes[i] /= len(runs)
    
for gen in gens:
    x.append(gen)

plt.title(f'Average max score over {len(runs)} runs for {envName}')
plt.xlabel('Generation')
plt.ylabel('Average Max Score')
plt.plot(x, average_maxes)
plt.show()

In [ ]:
test = 0
test += runs[0]['maxes'][49]
print(test)

In [ ]:
data = np.random.random(( 12 , 12 ))
plt.imshow( data , cmap = 'autumn' , interpolation = 'nearest' )
  
plt.title( "2-D Heat Map" )
plt.show()

In [ ]:
print([[b'u', b'l', b'l', b'd'],
           [b'u', b'u', b'u', b'd'],
           [b'u', b'u', b'd', b'd'],
           [b'u', b'r', b'r', b'u']], dtype='|S1')


In [ ]:
states = []
region = [1, 2, 0, 3]
# region = [0, 0, 0, 1]
# region = [1, 1, 0, 0]

# if region[3] - region[2] > 0:
for i in range(region[3] - region[2]+1):
    state = [0, 0]
    state[region[0]] = region[1]
    state[not region[0]] = region[2] + i
    states.append((state[0], state[1]))
# else:
#     state = [0, 0]
#     state[region[0]] = region[1]
#     state[not region[0]] = region[2]
#     states.append((state[0], state[1]))
    
print(states)